In [26]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

class VeggieVision:
    def __init__(self, input_shape=(224, 224, 3)):
        self.input_shape = input_shape
        self.build_model()

    def build_model(self):
        self.base_model = MobileNet(input_shape=self.input_shape, include_top=False, weights='imagenet')
        self.base_model.trainable = False

        self.model = models.Sequential([
            self.base_model,
            layers.Flatten(),
            layers.Dense(128, activation='relu'),
            layers.Dropout(0.3),
            layers.Dense(1, activation='linear')
        ])

    def augment_data(self, X_train):
        self.datagen = tf.keras.preprocessing.image.ImageDataGenerator(
            rotation_range= 360,
            horizontal_flip=True, 
            zoom_range=0.1, 
            width_shift_range=0.1, 
            height_shift_range=0.1, 
            brightness_range=[0.8, 1.2], 
            shear_range=0.1, 
            fill_mode='nearest'
        )
        self.datagen.fit(X_train)

    def fit(self, X, y, batch_size=32, epochs=20, learning_rate=1e-4, validation_split=0.2):
        
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=validation_split, random_state=42)
        self.augment_data(X_train)

        self.model.compile(optimizer=Adam(learning_rate=learning_rate), 
                           loss='mean_squared_error', metrics=['mae'])

        history = self.model.fit(self.datagen.flow(X_train, y_train, batch_size=batch_size),
                                 validation_data=(X_val, y_val), epochs=epochs)
        print(f'Validation MAE after initial training: {self.model.evaluate(X_val, y_val)[1]}')

        return history

    def fine_tune(self, X, y, batch_size=16, epochs=10, fine_tune_learning_rate=1e-5, n_unfreeze_layers=20, validation_split=0.2):
        
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=validation_split, random_state=42)

        for layer in self.base_model.layers[-n_unfreeze_layers:]:
            layer.trainable = True

        for layer in self.base_model.layers[:-n_unfreeze_layers]:
            layer.trainable = False

        self.model.compile(optimizer=Adam(learning_rate=fine_tune_learning_rate), 
                           loss='mean_squared_error', metrics=['mae'])

        history = self.model.fit(self.datagen.flow(X_train, y_train, batch_size=batch_size),
                                 validation_data=(X_val, y_val), epochs=epochs)
        print(f'Validation MAE after fine-tuning: {self.model.evaluate(X_val, y_val)[1]}')

        return history

    def evaluate(self, X_test, y_test):
        loss, mae = self.model.evaluate(X_test, y_test)
        print(f'Test Loss: {loss}, MAE: {mae}')

In [28]:
# Load the data
X = np.load("../data/processed_data/X.npy")
y = np.load("../data/processed_data/y.npy")

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
veggie_vision = VeggieVision()

In [10]:
veggie_vision.fit(X_train, y_train, learning_rate=1e-4)

Epoch 1/20
15/15 [==============================] - 12s 733ms/step - loss: 25674.0449 - mae: 81.7925 - val_loss: 5193.6929 - val_mae: 66.0893
Epoch 2/20
15/15 [==============================] - 11s 718ms/step - loss: 19248.1797 - mae: 60.1969 - val_loss: 2608.8069 - val_mae: 42.5443
Epoch 3/20
15/15 [==============================] - 11s 705ms/step - loss: 17216.1016 - mae: 47.7075 - val_loss: 1549.4004 - val_mae: 31.8311
Epoch 4/20
15/15 [==============================] - 11s 702ms/step - loss: 16081.5811 - mae: 38.6742 - val_loss: 1130.2761 - val_mae: 26.3144
Epoch 5/20
15/15 [==============================] - 11s 736ms/step - loss: 15165.7803 - mae: 34.2419 - val_loss: 952.8669 - val_mae: 23.9522
Epoch 6/20
15/15 [==============================] - 11s 721ms/step - loss: 15183.5010 - mae: 33.2929 - val_loss: 908.6908 - val_mae: 24.0614
Epoch 7/20
15/15 [==============================] - 11s 700ms/step - loss: 14403.2090 - mae: 32.1033 - val_loss: 872.8653 - val_mae: 23.8274
Epoch 8/2

KeyboardInterrupt: 

In [ ]:
veggie_vision.evaluate(X_test, y_test)